In [ ]:
# Install necessary libraries
!pip install kaggle opencv-python-headless

# Upload your Kaggle API token
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"srivarshc","key":"a89d7a34c8a6556f74febae47fc0302a"}'}

In [ ]:
# Create a Kaggle directory and move the Kaggle API token there
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset from Kaggle
!kaggle datasets download -d hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images

# Unzip the dataset
!unzip melanoma-skin-cancer-dataset-of-10000-images.zip


Dataset URL: https://www.kaggle.com/datasets/hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images
License(s): CC0-1.0
melanoma-skin-cancer-dataset-of-10000-images.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  melanoma-skin-cancer-dataset-of-10000-images.zip
replace melanoma_cancer_dataset/test/benign/melanoma_10000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tqdm import tqdm

In [ ]:
import os

# List the contents of the extracted dataset directory
!ls melanoma-skin-cancer-dataset-of-10000-images


ls: cannot access 'melanoma-skin-cancer-dataset-of-10000-images': No such file or directory


In [ ]:
import zipfile

# Specify the name of the uploaded dataset zip file
zip_file_name = 'melanoma-skin-cancer-dataset-of-10000-images.zip'

# Extract the dataset
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall('./')

# List the contents of the dataset directory
!ls


kaggle.json  melanoma_cancer_dataset  melanoma-skin-cancer-dataset-of-10000-images.zip	sample_data


In [ ]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from tqdm import tqdm

In [ ]:
# Load and preprocess dataset
def load_data(img_dir):
    images = []
    labels = []

    for label in ['benign', 'malignant']:
        label_dir = os.path.join(img_dir, 'train', label)
        for img_name in tqdm(os.listdir(label_dir)):
            img_path = os.path.join(label_dir, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (128, 128))  # Resize image to 128x128
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
                images.append(img)
                labels.append(1 if label == 'malignant' else 0)

    return np.array(images), np.array(labels)



In [ ]:
# Specify the directory where the dataset is extracted
img_dir = './melanoma_cancer_dataset/'
images, labels = load_data(img_dir)

100%|██████████| 4605/4605 [00:08<00:00, 541.65it/s]


In [ ]:
# Feature Extraction using HOG
def extract_hog_features(images):
    hog_features = []
    for img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        hog_feature = hog(gray, orientations=8, pixels_per_cell=(8, 8),
                          cells_per_block=(3, 3), visualize=False, transform_sqrt=True)
        hog_features.append(hog_feature)
    return np.array(hog_features)

# Extract HOG features from images
hog_features = extract_hog_features(images)



In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(hog_features, labels, test_size=0.2, random_state=42)



In [ ]:
# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.svm import LinearSVC

# Initialize and train LinearSVC model
svm = LinearSVC(C=1.0, loss='squared_hinge', max_iter=1000)
svm.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [ ]:
# Evaluate the model
y_pred = svm.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8209266007287871
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.83      0.83      1005
           1       0.81      0.81      0.81       916

    accuracy                           0.82      1921
   macro avg       0.82      0.82      0.82      1921
weighted avg       0.82      0.82      0.82      1921



In [ ]:
import cv2
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

In [ ]:
# Example data preprocessing
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Initialize and train LinearSVC model (example)
svm = LinearSVC(C=1.0, loss='squared_hinge', max_iter=1000)
svm.fit(X_train_scaled, y_train)


LinearSVC()

In [ ]:
# Define HOG feature extraction function
def extract_hog_features(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hog_features, _ = hog(gray, orientations=8, pixels_per_cell=(8, 8),
                          cells_per_block=(3, 3), visualize=True)
    return hog_features

In [ ]:
# Define classify_image function
def classify_image(img_path, model, scaler):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error: Unable to read image at {img_path}")
        return None

    img = cv2.resize(img, (128, 128))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    hog_feature = hog(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY), orientations=8, pixels_per_cell=(8, 8),
                      cells_per_block=(3, 3), visualize=False, transform_sqrt=True)
    hog_features = np.array([hog_feature])
    hog_features_scaled = scaler.transform(hog_features)

    prediction = model.predict(hog_features_scaled)
    return "Malignant" if prediction[0] == 1 else "Benign"




In [ ]:
# Example image path for prediction
example_img_path = './melanoma_cancer_dataset/train/benign/melanoma_1.jpg'
print("Prediction:", classify_image(example_img_path, svm, scaler))


Prediction: Benign


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Train LinearSVC model
svm = LinearSVC(C=1.0, loss='squared_hinge', max_iter=1000)
svm.fit(X_train, y_train)

# Evaluate the model on test set
y_pred = svm.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=['Benign', 'Malignant'])



Accuracy: 0.82
Classification Report:
              precision    recall  f1-score   support

      Benign       0.83      0.83      0.83      1005
   Malignant       0.81      0.81      0.81       916

    accuracy                           0.82      1921
   macro avg       0.82      0.82      0.82      1921
weighted avg       0.82      0.82      0.82      1921



In [ ]:
# Print metrics
print(f'Accuracy: {accuracy * 100:.2f}%')
print('Classification Report:')
print(report)


Accuracy: 82.09%
Classification Report:
              precision    recall  f1-score   support

      Benign       0.83      0.83      0.83      1005
   Malignant       0.81      0.81      0.81       916

    accuracy                           0.82      1921
   macro avg       0.82      0.82      0.82      1921
weighted avg       0.82      0.82      0.82      1921

